In [16]:
#%pip install crewai==0.119.0 crewai-tools==0.44.0 openai==1.25.1 google-cloud-bigquery==3.17.2 google-auth==2.29.0 google-auth-oauthlib==1.2.0 beautifulsoup4==4.12.3 playwright==1.44.0

In [4]:
import warnings

warnings.filterwarnings("ignore")

import getpass
import os
from typing import Optional

from crewai import Agent, Crew, Task
from crewai.tools import BaseTool
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from google.cloud import bigquery
from google.oauth2 import service_account

In [5]:

# Set the absolute path to your service account JSON file
json_path = "/Users/danielkim/_Capstone/PrepSense/adsp-34002-on02-prep-sense-ef1111b0833b.json"

# Verify the file exists
if not os.path.exists(json_path):
    raise FileNotFoundError(f"Service account JSON file not found at {json_path}")

# Set the environment variable
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = json_path

# Create credentials and client
credentials = service_account.Credentials.from_service_account_file(json_path)
bq_client = bigquery.Client(credentials=credentials, project=credentials.project_id)

print("Successfully connected to BigQuery!")

Successfully connected to BigQuery!


In [6]:
# #load openai api key
# import os
# import getpass
# os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

# dotenv to load environment variables

from dotenv import load_dotenv

load_dotenv()

# Initialize external tools



True

In [ ]:
os.environ["SERPER_API_KEY"] = getpass.getpass("Enter your Serper API key: ")

Enter your Serper API key:  ········


In [7]:
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o"

In [8]:
# Initialize external tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [9]:
from pydantic import BaseModel, Field


class BigQueryPantryToolInput(BaseModel):
    user_id: int = Field(..., description="The user ID for whom to fetch pantry items")
    pantry_table: Optional[str] = Field(None, description="The BigQuery pantry table")

In [10]:


class BigQueryPantryTool(BaseTool):
    name: str = "BigQueryPantryTool"
    description: str = "Fetches pantry items from BigQuery"
    args_schema: type[BaseModel] = BigQueryPantryToolInput

    def _run(self, user_id: int, pantry_table: Optional[str] = None) -> list:
        print(f"[DEBUG] Tool invoked with user_id={user_id}, pantry_table={pantry_table}")
        pantry_table = pantry_table or "adsp-34002-on02-prep-sense.Inventory.pantry_items"

        query = """
            SELECT
              p.user_id,
              pi.pantry_item_id,
              p.pantry_name,
              pp.product_name,
              pi.quantity,
              pi.used_quantity,
              pi.unit_of_measurement,
              pi.expiration_date,
              pi.created_at
            FROM `adsp-34002-on02-prep-sense.Inventory.pantry_items` AS pi
            JOIN `adsp-34002-on02-prep-sense.Inventory.pantry` AS p
              ON pi.pantry_id = p.pantry_id
            JOIN `adsp-34002-on02-prep-sense.Inventory.products` AS pp
              ON pi.pantry_item_id = pp.pantry_item_id
            WHERE p.user_id = @user_id
              AND pi.quantity > pi.used_quantity;
        """

        job_config = bigquery.QueryJobConfig(
            query_parameters=[
                bigquery.ScalarQueryParameter("user_id", "INT64", user_id)
            ]
        )

        try:
            query_job = bq_client.query(query, job_config=job_config)
            results = query_job.result()
            return [dict(row) for row in results]
        except Exception as e:
            return [{"error": str(e)}]

In [11]:
# testing for a specific user

bigquery_tool = BigQueryPantryTool()
results = bigquery_tool._run(user_id=102)
print(results)

[DEBUG] Tool invoked with user_id=102, pantry_table=None
[{'user_id': 102, 'pantry_item_id': 20004, 'pantry_name': 'kitchen_stock', 'product_name': 'rice', 'quantity': 2.0, 'used_quantity': 0, 'unit_of_measurement': 'kg', 'expiration_date': datetime.date(2025, 8, 10), 'created_at': datetime.datetime(2025, 5, 17, 9, 35, 40, 478671)}, {'user_id': 102, 'pantry_item_id': 20001, 'pantry_name': 'kitchen_stock', 'product_name': 'milk', 'quantity': 1.0, 'used_quantity': 0, 'unit_of_measurement': 'liter', 'expiration_date': datetime.date(2025, 8, 15), 'created_at': datetime.datetime(2025, 5, 17, 9, 35, 40, 478671)}, {'user_id': 102, 'pantry_item_id': 20005, 'pantry_name': 'kitchen_stock', 'product_name': 'vinegar', 'quantity': 6.0, 'used_quantity': 1, 'unit_of_measurement': 'ml', 'expiration_date': datetime.date(2025, 10, 1), 'created_at': datetime.datetime(2025, 5, 17, 9, 35, 40, 478671)}, {'user_id': 102, 'pantry_item_id': 20002, 'pantry_name': 'kitchen_stock', 'product_name': 'eggs', 'quanti

In [12]:
## Ingredient filter class

bq_client = bigquery.Client()

class IngredientFilterTool(BaseTool):
    name: str = "IngredientFilterTool"
    description: str = "Fetches non-expired and available pantry items for a given user from BigQuery"
    args_schema: type[BaseModel] = BigQueryPantryToolInput

    def _run(self, user_id: int, pantry_table: Optional[str] = None) -> list:
        print(f"[DEBUG] Tool invoked with user_id={user_id}, pantry_table={pantry_table}")
        pantry_table = pantry_table or "adsp-34002-on02-prep-sense.Inventory.pantry_items"

        query = """
            SELECT
              p.user_id,
              pi.pantry_item_id,
              p.pantry_name,
              pp.product_name,
              pi.quantity,
              pi.used_quantity,
              pi.unit_of_measurement,
              pi.expiration_date,
              pi.created_at
            FROM `adsp-34002-on02-prep-sense.Inventory.pantry_items` AS pi
            JOIN `adsp-34002-on02-prep-sense.Inventory.pantry` AS p
              ON pi.pantry_id = p.pantry_id
            JOIN `adsp-34002-on02-prep-sense.Inventory.products` AS pp
              ON pi.pantry_item_id = pp.pantry_item_id
            WHERE p.user_id = @user_id
              AND pi.quantity > pi.used_quantity
              AND pi.expiration_date >= CURRENT_DATE()
        """

        job_config = bigquery.QueryJobConfig(
            query_parameters=[
                bigquery.ScalarQueryParameter("user_id", "INT64", user_id)
            ]
        )

        try:
            query_job = bq_client.query(query, job_config=job_config)
            results = query_job.result()
            return [dict(row) for row in results]
        except Exception as e:
            return [{"error": str(e)}]

In [13]:
ingredient_tool = IngredientFilterTool()
results = ingredient_tool._run(user_id=102)
print(results)

[DEBUG] Tool invoked with user_id=102, pantry_table=None
[{'user_id': 102, 'pantry_item_id': 20004, 'pantry_name': 'kitchen_stock', 'product_name': 'rice', 'quantity': 2.0, 'used_quantity': 0, 'unit_of_measurement': 'kg', 'expiration_date': datetime.date(2025, 8, 10), 'created_at': datetime.datetime(2025, 5, 17, 9, 35, 40, 478671)}, {'user_id': 102, 'pantry_item_id': 20001, 'pantry_name': 'kitchen_stock', 'product_name': 'milk', 'quantity': 1.0, 'used_quantity': 0, 'unit_of_measurement': 'liter', 'expiration_date': datetime.date(2025, 8, 15), 'created_at': datetime.datetime(2025, 5, 17, 9, 35, 40, 478671)}, {'user_id': 102, 'pantry_item_id': 20005, 'pantry_name': 'kitchen_stock', 'product_name': 'vinegar', 'quantity': 6.0, 'used_quantity': 1, 'unit_of_measurement': 'ml', 'expiration_date': datetime.date(2025, 10, 1), 'created_at': datetime.datetime(2025, 5, 17, 9, 35, 40, 478671)}, {'user_id': 102, 'pantry_item_id': 20002, 'pantry_name': 'kitchen_stock', 'product_name': 'eggs', 'quanti

In [15]:
class UserRestrictionToolInput(BaseModel):
    user_id: int

class UserRestrictionTool(BaseTool):
    name: str = "UserPreferenceTool"
    description: str = "Fetches dietary restrictions and allergens for a user"
    args_schema: type[BaseModel] = UserRestrictionToolInput

    def _run(self, user_id: int) -> dict:
        query = """
            SELECT dietary_preference, allergens
            FROM `adsp-34002-on02-prep-sense.Inventory.user_preference`
            WHERE user_id = @user_id
        """

        job_config = bigquery.QueryJobConfig(
            query_parameters=[
                bigquery.ScalarQueryParameter("user_id", "INT64", user_id)
            ]
        )

        try:
            query_job = bq_client.query(query, job_config=job_config)
            result = list(query_job.result())
            if result:
                return dict(result[0])
            else:
                return {"message": "No dietary data found for this user."}
        except Exception as e:
            return {"error": str(e)}

In [16]:
user_restriction_tool = UserRestrictionTool()
results = user_restriction_tool._run(user_id=102)
print(results)

{'dietary_preference': ['vegetarian', 'low-carb'], 'allergens': ['gluten']}


In [18]:
# Define AI Agents for Pantry-to-Dinner Workflow
pantry_scan_agent = Agent(
    role="Pantry Scan Agent",
    goal="Retrieve list of available ingredients for user ID {user_id} from BigQuery pantry table",
    tools=[bigquery_tool],
    verbose=True,
    backstory="You have access to structured data sources like BigQuery and can extract relevant pantry contents."
)

ingredient_filter_agent = Agent(
    role="Ingredient Filter Agent",
    goal="Filter out expired or unusable items from the pantry list",
    tools=[ingredient_tool],
    verbose=True,
    backstory="You ensure only safe and usable ingredients are passed to recipe generation agents."
)

recipe_search_agent = Agent(
    role="Recipe Search Agent",
    goal="Find recipes that can be made using the filtered ingredients",
    tools=[search_tool, scrape_tool, ingredient_tool],
    verbose=True,
    backstory="You search online and through structured recipe repositories to find viable dinner options."
)

nutritional_agent = Agent(
    role="Nutritional Agent",
    goal="Evaluate the nutritional balance of each proposed recipe",
    tools=[search_tool],
    verbose=True,
    backstory="You ensure that dinner suggestions are healthy and balanced. Give me a detailed nutritional breakdown."
)

user_restriction_tool = UserRestrictionTool()  # Instantiate the tool

user_preferences_agent = Agent(
    role="User Preferences Agent",
    goal="Filter out ingredients and recipes that conflict with the user's dietary restrictions and allergens",
    tools=[user_restriction_tool],  # Pass the instance
    verbose=True,
    backstory="You ensure the user only sees recipes and ingredients that comply with their health and dietary requirements."
)

recipe_scoring_agent = Agent(
    role="Recipe Scoring Agent",
    goal="Score recipes based on match with ingredients, nutrition, and user preferences",
    tools=[],
    verbose=True,
    backstory="You rank suggestions to make it easier for users to choose."
)

response_formatting_agent = Agent(
    role="Response Formatting Agent",
    goal="Format and summarize the best recipe suggestions for user-friendly viewing",
    tools=[],
    verbose=True,
    backstory="You turn results into clean summaries, including ingredients and nutrition info."
)

In [21]:
# Define Tasks for Agents

def create_task(agent, description, expected_output, input_variables=None):
    print(f"[DEBUG] Creating task for {agent.role} with inputs: {input_variables}")
    return Task(
        description=description,
        expected_output=expected_output,
        human_input=False,
        agent=agent,
        input_variables=input_variables or []
    )

In [22]:
tasks = [
    create_task(
        pantry_scan_agent,
        "Query BigQuery table for usable ingredients for a given user_id",
        "List of pantry ingredients for a given user_id",
        input_variables=["user_id", "pantry_table"]
    ),
        create_task(
        ingredient_filter_agent,
        "Filter out expired or unusable items from the user's pantry list based on expiration date and usage",
        "List of usable (non-expired) pantry ingredients for a given user_id",
        input_variables=["user_id", "pantry_table"]
    ),
    create_task(recipe_search_agent, "Find dinner recipes that use these ingredients", "List of at least 5 suitable recipes"),
    create_task(nutritional_agent, "Evaluate each recipe for nutritional value", "Nutritional score and comments for each recipe"),
    create_task(
        user_preferences_agent,
        "Filter recipes based on user dietary preferences",
        "List of safe ingredients or recipes based on user dietary restrictions",
        input_variables=["user_id"]
    ),
    create_task(recipe_scoring_agent, "Score recipes on match, nutrition, and user preference", "Ranked list of recipes"),
    create_task(response_formatting_agent, "Format top 3 recipes with ingredients, instructions, and nutrition summary", "Formatted response in markdown")
]

[DEBUG] Creating task for Pantry Scan Agent with inputs: ['user_id', 'pantry_table']
[DEBUG] Creating task for Ingredient Filter Agent with inputs: ['user_id', 'pantry_table']
[DEBUG] Creating task for Recipe Search Agent with inputs: None
[DEBUG] Creating task for Nutritional Agent with inputs: None
[DEBUG] Creating task for User Preferences Agent with inputs: ['user_id']
[DEBUG] Creating task for Recipe Scoring Agent with inputs: None
[DEBUG] Creating task for Response Formatting Agent with inputs: None


In [25]:
# Ensure 'tasks' is defined before using it
try:
    tasks
except NameError:
    raise RuntimeError("The variable 'tasks' is not defined. Please run the cell that defines 'tasks' before running this cell.")

# Create and run the Crew
pantry_dinner_crew = Crew(
    agents=[
        pantry_scan_agent,
        ingredient_filter_agent,
        recipe_search_agent,
        nutritional_agent,
        user_preferences_agent,
        recipe_scoring_agent,
        response_formatting_agent
    ],
    tasks=tasks,
    verbose=True
)

inputs = {
 #   "user_dietary_preferences": "vegetarian, no nuts",
    "pantry_table": "adsp-34002-on02-prep-sense.Inventory.pantry_items",
    "user_id":102
}

In [24]:
print("[DEBUG] Final Inputs to Crew:", inputs)
result = pantry_dinner_crew.kickoff(inputs=inputs)
print(result)

[DEBUG] Final Inputs to Crew: {'pantry_table': 'adsp-34002-on02-prep-sense.Inventory.pantry_items', 'user_id': 102}


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 1c061468-1698-4bd4-a7d5-34539175a669                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 390dda2a-1600-4c2a-b504-ad48d114423e
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 390dda2a-1600-4c2a-b504-ad48d114423e
       Status: Executing Task...
    └── 🤖 Agent: Pantry Scan Agent
            Status: In Progress

# Agent: Pantry Scan Agent
## Task: Query BigQuery table for usable ingredients for a given user_id


...

🚀 Crew: crew
├── 📋 Task: 390dda2a-1600-4c2a-b504-ad48d114423e
│      Assigned to: Pantry Scan Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Pantry Scan Agent
│           Status: ✅ Completed
├── 📋 Task: 16511ceb-34f2-4122-821c-11639e7019d9
│      Assigned to: Ingredient Filter Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Ingredient Filter Agent
│           Status: ✅ Completed
├── 📋 Task: 6fa5eea2-a63a-4543-8481-d7c9fd219380
│      Assigned to: Recipe Search Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Recipe Search Agent
│           Status: ✅ Completed
├── 📋 Task: 95b9137c-e659-4761-b44f-d91edd78628e
│      Assigned to: Nutritional Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Nutritional Agent
│           Status: ✅ Completed
├── 📋 Task: bf62fa00-eb5b-4dc5-ae82-93b33da8a90d
│      Assigned to: User Preferences Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: User Preferences Agent
│           Status: ✅ Completed
├── 📋 Task: f0603d49-0405-45c0-b6c1-ba91c1caaecc
│      Assigned to: Recipe Scoring Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Recipe Scoring Agent
│           Status: ✅ Completed
└── 📋 Task: 7ecbbb4c-8337-4619-a446-ea79631421e1
       Status: Executing Task...
    └── 🤖 Agent: Response Formatting Agent
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 390dda2a-1600-4c2a-b504-ad48d114423e
│      Assigned to: Pantry Scan Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Pantry Scan Agent
│           Status: ✅ Completed
├── 📋 Task: 16511ceb-34f2-4122-821c-11639e7019d9
│      Assigned to: Ingredient Filter Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Ingredient Filter Agent
│           Status: ✅ Completed
├── 📋 Task: 6fa5eea2-a63a-4543-8481-d7c9fd219380
│      Assigned to: Recipe Search Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Recipe Search Agent
│           Status: ✅ Completed
├── 📋 Task: 95b9137c-e659-4761-b44f-d91edd78628e
│      Assigned to: Nutritional Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Nutritional Agent
│           Status: ✅ Completed
├── 📋 Task: bf62fa00-eb5b-4dc5-ae82-93b33da8a90d
│      Assigned to: User Preferences Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: User Preferences Agent
│           Status: ✅ Completed
├── 📋 Task: f0603d49-0405-45c0-b6c1-ba91c1caaecc
│      Assigned to: Recipe Scoring Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Recipe Scoring Agent
│           Status: ✅ Completed
└── 📋 Task: 7ecbbb4c-8337-4619-a446-ea79631421e1
       Assigned to: Response Formatting Agent
       Status: ✅ Completed
    └── 🤖 Agent: Response Formatting Agent
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 7ecbbb4c-8337-4619-a446-ea79631421e1                                                                     │
│  Agent: Response Formatting Agent                                                                               │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 1c061468-1698-4bd4-a7d5-34539175a669                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Below are the top 3 recipes formatted with ingredients, instructions, and a nutritional summary for user-friendly viewing:

### 1. Oyakodon (Japanese Chicken and Egg Rice Bowl)
- **Ingredients:**
  - 2 cups cooked rice
  - 2 eggs
  - 1 chicken breast (optional if using pantry proteins)
  - 1/4 cup soy sauce
  - 1/4 cup mirin (or substitute with a mix of vinegar and sugar)
  - 1 onion, thinly sliced
  - Optional: chopped green onions for garnish

- **Instructions:**
  1. Cook the onion slices in a skillet over medium heat until soft.
  2. Add the chicken breast slices (if using) and cook until the chicken is done.
  3. Pour in the soy sauce and mirin, stirring to combine.
  4. Beat the eggs in a bowl and pour over the cooked ingredients in the skillet.
  5. Cover the skillet and let the eggs cook until just set.
  6. Serve over a bowl of hot rice and garnish with chopped green onions.

- **Nutritional Summary:**
  - Calories: Approximately 510-762 per serving
  - Protein: 32-53g
  - Car

In [26]:
# Helper functions

def get_user_pantry_items(user_id: int):
    query = f"""
        SELECT *
        FROM `adsp-34002-on02-prep-sense.Inventory.pantry_items`
        WHERE pantry_id = {user_id} AND quantity > 0
    """
    try:
        query_job = bq_client.query(query)
        results = query_job.result()
        return [dict(row) for row in results]
    except Exception as e:
        return [{"error": str(e)}]

def insert_recipe(recipe_data: dict):
    table_id = "adsp-34002-on02-prep-sense.Inventory.recipies"
    errors = bq_client.insert_rows_json(table_id, [recipe_data])
    if errors == []:
        print("✅ New recipe added.")
    else:
        print("❌ Errors:", errors)